<a href="https://colab.research.google.com/github/RuslanMavlitov/IDE/blob/master/AvitoDemandSampleEmpty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Новый раздел

# Новый раздел

In [ ]:
ls

sample_data/


In [2]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split

import lightgbm as lgb

In [3]:
data = pd.read_csv("/content/drive/MyDrive/SF/data/train.csv.zip", parse_dates=["activation_date"])
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [ ]:
# Вы можете сделать EDA, например, посмотреть deal probability по категориям. Если вам интересно =)

In [4]:
  train, test = train_test_split(data, random_state=13, test_size=0.3)
  val, test = train_test_split(test, random_state=13, test_size=0.3)

In [5]:
# Давайте обучим простенькую модель

# Отделите целевую переменную
y_train = train['deal_probability']
y_val = val['deal_probability']
y_test = test['deal_probability']

# Давайте добавим фактор: день недели активации
train["activation_weekday"] = train['activation_date'].dt.dayofweek
val["activation_weekday"] = val['activation_date'].dt.dayofweek
test["activation_weekday"] = test['activation_date'].dt.dayofweek

# Давайте заэнкодим категориальные переменные
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3"]
for col in cat_vars:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(val[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    val[col] = lbl.transform(list(val[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))

cols_to_drop = ["item_id", "user_id", "title", "description", "activation_date", "image", "deal_probability"]
X_train = train.drop(cols_to_drop, axis=1)
X_val = val.drop(cols_to_drop, axis=1)
X_test = test.drop(cols_to_drop, axis=1)

In [6]:
X_val.head()

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,user_type,image_top_1,activation_weekday
128604,15,1153,4,29,126,203,143,550.0,8,1,377.0,0
839577,5,329,4,29,207,193,133,2500.0,66,1,46.0,1
230058,10,743,5,15,284,188,0,13000.0,61,1,2218.0,1
153103,16,1276,6,0,278,100,665,3470000.0,1,1,1118.0,2
1063751,16,1063,4,29,126,122,149,5500.0,69,0,536.0,3


In [7]:
params = {
    "objective" : "regression",
    "metric" : "rmse",
    "num_leaves" : 30,
    "learning_rate" : 0.1,
    "bagging_fraction" : 0.7,
    "feature_fraction" : 0.7,
    "bagging_frequency" : 5,
    "bagging_seed" : 2018,
    "verbosity" : -1
}
    
lgtrain = lgb.Dataset(X_train, label=y_train) # Создать lgb Dataset с тренировочной выборкой
lgval =  lgb.Dataset(X_val, label=y_val) # Создать lgb Dataset с валидационной выборкой
model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=20)

pred_test_y = model.predict(X_test, num_iteration=model.best_iteration)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: bagging_frequency
Training until validation scores don't improve for 100 rounds
[20]	valid_0's rmse: 0.236582
[40]	valid_0's rmse: 0.233712
[60]	valid_0's rmse: 0.232541
[80]	valid_0's rmse: 0.231955
[100]	valid_0's rmse: 0.231508
[120]	valid_0's rmse: 0.231172
[140]	valid_0's rmse: 0.230845
[160]	valid_0's rmse: 0.230558
[180]	valid_0's rmse: 0.230358
[200]	valid_0's rmse: 0.230217
[220]	valid_0's rmse: 0.230054
[240]	valid_0's rmse: 0.229902
[260]	valid_0's rmse: 0.229783
[280]	valid_0's rmse: 0.229633
[300]	valid_0's rmse: 0.229514
[320]	valid_0's rmse: 0.229363
[340]	valid_0's rmse: 0.229257
[360]	valid_0's rmse: 0.229162
[380]	valid_0's rmse: 0.229071
[400]	valid_0's rmse: 0.229002
[420]	valid_0's rmse: 0.228929
[440]	valid_0's rmse: 0.228867
[460]	valid_0's rmse: 0.228819
[480]	valid_0's rmse: 0.228769
[500]	valid_0's rmse: 0.228719
[520]	valid_0's rmse: 0.228663
[540]	valid_0's rmse: 0.228621
[560]	valid_0's rmse: 0.228583
[580]	valid_0's 

In [8]:
print(metrics.mean_squared_error(y_test, pred_test_y))

0.05178713838770174
